In [1]:
import experiments_sbm as experiments_sbm
import numpy as np
import igraph as ig
import scipy as sp

In [2]:
n = 200000
n_clusters = 2
alpha = [ 1 / n_clusters for i in range( n_clusters ) ]

block_sizes = [ int( alpha[i] * n ) for i in range( n_clusters ) ]

B = np.array( [ [1,2],[2,8] ])
asymptotic_rate = np.log(n) / n 
lambdain = 1
lambdaout = 1
weights_parameters = (lambdain - lambdaout) * np.eye( n_clusters ) + lambdaout * np.ones( n_clusters )

In [4]:
G = ig.Graph.SBM( np.sum(block_sizes), B * asymptotic_rate, block_sizes, directed=False, loops=False)
G = experiments_sbm.add_weights( G, weights_parameters, block_sizes, weight_distribution = 'exponential' )

In [ ]:
source_vertices = [ i for i in range( block_sizes[0]) ]
target_vertices = [ i for i in range( block_sizes[0]+1, block_sizes[0]+block_sizes[1] ) ]

n_pairs_sampled = 100

costs_mean = np.zeros((2,2))
costs_ste = np.zeros((2,2))

costs_mean[0,0], costs_ste[0,0] = experiments_sbm.estimateCostShortestPath( G, n_pairs_sampled, source_vertices, source_vertices )
costs_mean[0,1], costs_ste[0,1] = experiments_sbm.estimateCostShortestPath( G, n_pairs_sampled, source_vertices, target_vertices )
costs_mean[1,1], costs_ste[1,1] = experiments_sbm.estimateCostShortestPath( G, n_pairs_sampled, target_vertices, target_vertices )
costs_mean[1,0], costs_ste[1,0] = experiments_sbm.estimateCostShortestPath( G, n_pairs_sampled, target_vertices, source_vertices )


100%|██████████| 100/100 [00:21<00:00,  4.55it/s]


In [6]:
costs_mean

array([[0.36018604, 0.291019  ],
       [0.29631385, 0.23779639]])

In [1]:
costs_std

NameError: name 'costs_std' is not defined

In [7]:
T = np.multiply( weights_parameters, B) @ np.diag( alpha )
vals, vecs = np.linalg.eigh(T)
rho = np.max( vals )

gumbels = sp.stats.gumbel_r.rvs( size = 1000 ) + sp.stats.gumbel_r.rvs( size = 1000 ) - sp.stats.gumbel_r.rvs( size = 1000 )

In [8]:
rescaled_costs00 = experiments_sbm.distributionalLimitRescaledCosts( G, 100, source_vertices, source_vertices, rho, verbose = True ) 
sp.stats.ks_2samp(rescaled_costs00, gumbels )

100%|██████████| 100/100 [00:21<00:00,  4.56it/s]


KstestResult(statistic=np.float64(0.559), pvalue=np.float64(4.830800899121636e-27), statistic_location=np.float64(3.9359237962656035), statistic_sign=np.int8(-1))

In [9]:
rescaled_costs01 = experiments_sbm.distributionalLimitRescaledCosts( G, 100, source_vertices, source_vertices, rho, verbose = True ) 
sp.stats.ks_2samp( rescaled_costs01, gumbels )

100%|██████████| 100/100 [00:22<00:00,  4.41it/s]


KstestResult(statistic=np.float64(0.628), pvalue=np.float64(8.612768332455424e-35), statistic_location=np.float64(4.159270610527142), statistic_sign=np.int8(-1))